In [32]:
%run Model_setup.ipynb
%run Create_list_of_URLs.ipynb

['www.free.fr', 'www.google.fr', 'www.amazon.fr', 'www.lemonde.fr', 'www.lefigaro.fr', 'www.weborama.fr', 'www.orange.fr', 'www.leboncoin.fr', 'www.rfi.fr', 'www.francetvinfo.fr', 'www.cnrs.fr']


In [36]:
# Create list of URLs of specific dates
dictory_of_urls = go_over_websites(first_ten_websites[1:])

www.google.fr https://web.archive.org/web/20200712193240/http://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20200812202822/http://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20200912194125/http://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20201012193024/http://www.google.fr Success.
www.google.fr https://web.archive.org/web/20201112214452/https://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20201212193814/https://www.google.fr/ Success.
www.amazon.fr https://web.archive.org/web/20200712220603/https://www.amazon.fr/ Success.
www.amazon.fr https://web.archive.org/web/20200812191533/https://www.amazon.fr/ Success.
www.amazon.fr https://web.archive.org/web/20200912173925/https://www.amazon.fr/ Success.
www.amazon.fr https://web.archive.org/web/20201012192720/https://www.amazon.fr/ Success.
www.amazon.fr https://web.archive.org/web/20201113044348/https://www.amazon.fr/ Success.
www.amazon.fr https://web.

TooManyRequestsError: Can not save 'www.lemonde.fr'. Save request refused by the server. Save Page Now limits saving 15 URLs per minutes. Try waiting for 5 minutes and then try again.

In [ ]:
def crawling(url):
    """
    Main function to extract iframes and active elements from a web page and pass them to an ML model.

    Args:
        url (str): The URL of the web page.
    """
    driver = setup_firefox_driver()
    driver.get(url)
    sleep(5)
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    
    iframes = soup.find_all(['div', 'iframe'])
    active_elements = soup.find_all(['a', 'button', 'input', 'yt-formatted-string', 'p', 'span', 'form'])
    
    #button_elements = soup.find_all(['button', 'input', 'span', 'a'])
    button_elements = soup.find_all(['button', 'input[type="submit"]', 'a'])
    

    # Extract text content from iframes and active elements
    text_content = extract_text_from_elements(active_elements)
        
    button_text = extract_text_from_elements(button_elements)
    #text_content.extend(extract_text_from_iframes(iframes, driver))
    
    #check = search_dialog(iframes, text_content)
    search_frames(iframes, text_content, button_text)
    
    driver.quit()
    dialog_result = pass_to_ml_model_dialog(text_content)
    if dialog_result != "not found":
        return (dialog_result, pass_to_ml_model_buttons(button_text))
    return "not found"

In [ ]:
webs = []
for web in dictory_of_urls:
    webs.append(web)

In [ ]:
for web in webs:
    for web_visit in dictory_of_urls[web]:
        results_web = main(web_visit[0])
        if not ( results_web == "not found"):
            print(web_visit)
            print(results_web)
            break